# Cleaning Process Documentation - Hackathon Project

### Objective
Clean, transform, and prepare the *Online Retail* dataset for analysis by removing errors, correcting inconsistencies, and structuring the data for visualisation and insights.

---

### Credits
Tableau Prep flow was used alongside this notebook for quick data discovery and drill down

---

In [128]:
import pandas as pd

df = pd.read_csv(r'../data/online_retail.csv')

#### Create a Data Dictionary of the imported dataset 

In [129]:
# Custom Function to create a comprehensive data dictionary for Online Retail datasets
# Takes a DataFrame and returns a data dictionary
def create_data_dictionary(df):
    # Official descriptions from UCI ML Repository Variables Table: https://archive.ics.uci.edu/dataset/352/online+retail
    descriptions = {
        'InvoiceNo': 'Invoice number - 6-digit integral number uniquely assigned to each transaction. If starts with letter "c", indicates a cancellation',
        'StockCode': 'Product (item) code - 5-digit integral number uniquely assigned to each distinct product',
        'Description': 'Product (item) name/description',
        'Quantity': 'The quantities of each product (item) per transaction',
        'InvoiceDate': 'Invoice date and time - the day and time when each transaction was generated',
        'UnitPrice': 'Unit price - product price per unit in sterling (£)',
        'CustomerID': '5-digit integral number uniquely assigned to each customer',
        'Country': 'Country name - the name of the country where each customer resides'
    }
    
    dictionary_data = []
    for column in df.columns:
        # Get 3 sample values (non-null)
        sample_values = df[column].dropna().head(3).tolist()
        sample_str = ', '.join([str(x) for x in sample_values])
        
        dictionary_data.append({
            'Column': column,
            'Data Type': str(df[column].dtype),
            'Missing Values': df[column].isnull().sum(),
            'Missing %': round((df[column].isnull().sum() / len(df)) * 100, 2),
            'Unique Values': df[column].nunique(),
            'Sample Values': sample_str,
            'Description': descriptions.get(column, 'Custom/Engineered column - description needed')
        })
    return pd.DataFrame(dictionary_data)

# Store the dictionary in a variable
raw_data_dictionary = create_data_dictionary(df)

# Display data dictionary
raw_data_dictionary


,Column,Data Type,Missing Values,Missing %,Unique Values,Sample Values,Description
0,InvoiceNo,object,0,0.00,25900,"536365, 536365, 536365",Invoice number - 6-digit integral number uniqu...
1,StockCode,object,0,0.00,4070,"85123A, 71053, 84406B",Product (item) code - 5-digit integral number ...
2,Description,object,1454,0.27,4223,"WHITE HANGING HEART T-LIGHT HOLDER, WHITE META...",Product (item) name/description
3,Quantity,int64,0,0.00,722,"6, 6, 8",The quantities of each product (item) per tran...
4,InvoiceDate,object,0,0.00,23260,"2010-12-01 08:26:00, 2010-12-01 08:26:00, 2010...",Invoice date and time - the day and time when ...
5,UnitPrice,float64,0,0.00,1630,"2.55, 3.39, 2.75",Unit price - product price per unit in sterlin...
6,CustomerID,int64,0,0.00,4372,"17850, 17850, 17850",5-digit integral number uniquely assigned to e...
7,Country,object,0,0.00,38,"United Kingdom, United Kingdom, United Kingdom",Country name - the name of the country where e...


#### Check Invoice Column

- We have used tableau prep to quickly identify data conserns, we first noticed that InvoiceNo contains string values.

In [130]:
def analyse_invoice_column(df, column_name='InvoiceNo'):
    """
    Analyse the InvoiceNo column to identify different invoice types
    
    Parameters:
    df (pandas.DataFrame): The dataframe to analyze
    column_name (str): The name of the invoice column to analyze (default: 'InvoiceNo')
    
    Prints analysis results including counts and sample values for regular and non-standard invoices.
    """
    print(f"Invoice Column records: {len(df)}")
    
    # Check for different invoice patterns
    regular_invoices = df[df[column_name].astype(str).str.match(r'^\d{6}$')]
    non_standard_invoices = df[~df[column_name].astype(str).str.match(r'^\d{6}$')]
    
    print(f"\nRegular invoices (exactly 6 digits): {len(regular_invoices)} ({len(regular_invoices)/len(df)*100:.1f}%)")
    print(f"Non-standard invoices (not exactly 6 digits): {len(non_standard_invoices)} ({len(non_standard_invoices)/len(df)*100:.1f}%)")
    
    # Show sample invoice numbers for each type
    print("\nExpected invoice format:")
    print(regular_invoices[column_name].head(5).tolist())
    
    if len(non_standard_invoices) > 0:
        print("\nInvoices with string values:")
        print(non_standard_invoices[column_name].head(10).tolist())

# Run the analysis on the current dataset
analyse_invoice_column(df)

Invoice Column records: 541909

Regular invoices (exactly 6 digits): 532618 (98.3%)
Non-standard invoices (not exactly 6 digits): 9291 (1.7%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']

Invoices with string values:
['C536379', 'C536383', 'C536391', 'C536391', 'C536391', 'C536391', 'C536391', 'C536391', 'C536391', 'C536506']

Regular invoices (exactly 6 digits): 532618 (98.3%)
Non-standard invoices (not exactly 6 digits): 9291 (1.7%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']

Invoices with string values:
['C536379', 'C536383', 'C536391', 'C536391', 'C536391', 'C536391', 'C536391', 'C536391', 'C536391', 'C536506']


### Filter out rows where InvoiceNo contains 'C' (cancellations)
- Dataset research from {[link](https://archive.ics.uci.edu/dataset/352/online+retail)} shows that invoice numbers begining with C are cancellations, these are not within the scope of our user stories and will be removed from the dataset.

In [131]:
df = df[~df['InvoiceNo'].astype(str).str.contains('C', case=False, na=False)]
analyse_invoice_column(df)

Invoice Column records: 532621

Regular invoices (exactly 6 digits): 532618 (100.0%)
Non-standard invoices (not exactly 6 digits): 3 (0.0%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']

Invoices with string values:
['A563185', 'A563186', 'A563187']

Regular invoices (exactly 6 digits): 532618 (100.0%)
Non-standard invoices (not exactly 6 digits): 3 (0.0%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']

Invoices with string values:
['A563185', 'A563186', 'A563187']


### Filter out rows where InvoiceNo contains 'A' (Adjust bad debt)
- Dataset research from tableau prep shows that A records all have the description "Adjust bad debt" which are outside of the scope of our analysis and will be removed.

In [132]:
df = df[~df['InvoiceNo'].astype(str).str.contains('A', case=False, na=False)]
analyse_invoice_column(df)

Invoice Column records: 532618

Regular invoices (exactly 6 digits): 532618 (100.0%)
Non-standard invoices (not exactly 6 digits): 0 (0.0%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']

Regular invoices (exactly 6 digits): 532618 (100.0%)
Non-standard invoices (not exactly 6 digits): 0 (0.0%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']


### Filter out rows with zero or negative quantities
- Zero and negative quantities typically represent stock adjustments, write-offs, or inventory corrections rather than actual sales transactions
- These records are not relevant for our retail transaction analysis as they don't represent customer purchases
- Removing these ensures we focus only on genuine sales data for our analysis

In [133]:
df = df[df['Quantity'] > 0]

### Filter out adjustments and write offs
- Exploring the data with Tableau Prep has revealed that ajustments and write off actions appear to be hand typed into the product description feild, the product description is otherwise standardised to be in upper case except for cases where a measurement is included in the product name such as: cm, g, 45x45.

In [134]:
# Filter out rows where Description doesn't have at least 2 consecutive uppercase characters
# This removes manual adjustments and write-offs that are typically lowercase or mixed case
df = df[df['Description'].str.contains(r'[A-Z]{2,}', na=False)]

print(f"Rows remaining after filtering descriptions: {len(df)}")

Rows remaining after filtering descriptions: 530071


### Check for nulls after initial cleaning steps to check quality

In [135]:
# Check for null values in the cleaned dataset
print("Null values in cleaned dataset:")
print(df.isnull().sum())

Null values in cleaned dataset:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


### Check for duplicate rows

In [136]:
# Check for duplicates using the same columns that will be used for removal
duplicates_check = df.duplicated(subset=['InvoiceNo', 'StockCode', 'Quantity', 'CustomerID']).sum()
print(f"Duplicate invoice lines (excluding InvoiceDate): {duplicates_check}")

# Also check with InvoiceDate for comparison
duplicates_with_date = df.duplicated(subset=['InvoiceNo', 'StockCode', 'Quantity', 'CustomerID', 'InvoiceDate']).sum()
print(f"Duplicate invoice lines (including InvoiceDate): {duplicates_with_date}")

# Show the difference
print(f"Difference: {duplicates_check - duplicates_with_date}")

Duplicate invoice lines (excluding InvoiceDate): 5296
Duplicate invoice lines (including InvoiceDate): 5294
Difference: 2


This duplicate check validates if a duplicate order is a repeat order or if the database has two records for the same order, here we find no duplciates that need to be removed.

### Missing or unspecified countries

In [137]:
# Check for missing countries and 'Unspecified' entries
missing_countries = df['Country'].isnull().sum()
unspecified_countries = (df['Country'] == 'Unspecified').sum()
total_records = len(df)

print(f"Unspecified countries: {unspecified_countries} ({unspecified_countries/total_records*100:.2f}%)")
print(f"Total problematic country records: {missing_countries + unspecified_countries} ({(missing_countries + unspecified_countries)/total_records*100:.2f}%)")

Unspecified countries: 446 (0.08%)
Total problematic country records: 446 (0.08%)


Some orders are assigned to Unspecified countries, for our analysis we want to account for the location of our orders for our geographic analysis and these make up less than 0.1 % of the dataset.

In [138]:
df = df[df['Country'].notna()]
df = df[df['Country'] != 'Unspecified']

### Re-analyze Invoice Column After Cleaning
Now let's check the invoice column again after all our cleaning steps to see how the data looks.

In [139]:
# Use the function to re-analyze the invoice column after cleaning
print("=== AFTER CLEANING ===")
analyse_invoice_column(df)

=== AFTER CLEANING ===
Invoice Column records: 529625

Regular invoices (exactly 6 digits): 529625 (100.0%)
Non-standard invoices (not exactly 6 digits): 0 (0.0%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']

Regular invoices (exactly 6 digits): 529625 (100.0%)
Non-standard invoices (not exactly 6 digits): 0 (0.0%)

Expected invoice format:
['536365', '536365', '536365', '536365', '536365']


### Convert InvoiceNo to Int

In [141]:
# Convert InvoiceNo from object to integer
df['InvoiceNo'] = df['InvoiceNo'].astype(int)

print(f"InvoiceNo data type after conversion: {df['InvoiceNo'].dtype}")
print(f"Sample InvoiceNo values: {df['InvoiceNo'].head().tolist()}")

InvoiceNo data type after conversion: int64
Sample InvoiceNo values: [536365, 536365, 536365, 536365, 536365]


### Convert InvoiceDate to DateTime

In [145]:
# Convert InvoiceDate from object (string) to datetime for proper time-based analysis
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print(f"InvoiceDate data type after conversion: {df['InvoiceDate'].dtype}")
print(f"Sample InvoiceDate values: {df['InvoiceDate'].head().tolist()}")
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")

InvoiceDate data type after conversion: datetime64[ns]
Sample InvoiceDate values: [Timestamp('2010-12-01 08:26:00'), Timestamp('2010-12-01 08:26:00'), Timestamp('2010-12-01 08:26:00'), Timestamp('2010-12-01 08:26:00'), Timestamp('2010-12-01 08:26:00')]
Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00


#### Create a Data Dictionary of the cleaned dataset 

In [146]:
# Store the dictionary in a variable
clean_data_dictionary = create_data_dictionary(df)

# Display data dictionary
clean_data_dictionary

,Column,Data Type,Missing Values,Missing %,Unique Values,Sample Values,Description
0,InvoiceNo,int64,0,0.0,19862,"536365, 536365, 536365",Invoice number - 6-digit integral number uniqu...
1,StockCode,object,0,0.0,3908,"85123A, 71053, 84406B",Product (item) code - 5-digit integral number ...
2,Description,object,0,0.0,4016,"WHITE HANGING HEART T-LIGHT HOLDER, WHITE META...",Product (item) name/description
3,Quantity,int64,0,0.0,376,"6, 6, 8",The quantities of each product (item) per tran...
4,InvoiceDate,datetime64[ns],0,0.0,18416,"2010-12-01 08:26:00, 2010-12-01 08:26:00, 2010...",Invoice date and time - the day and time when ...
5,UnitPrice,float64,0,0.0,1211,"2.55, 3.39, 2.75",Unit price - product price per unit in sterlin...
6,CustomerID,int64,0,0.0,4331,"17850, 17850, 17850",5-digit integral number uniquely assigned to e...
7,Country,object,0,0.0,37,"United Kingdom, United Kingdom, United Kingdom",Country name - the name of the country where e...


### Save the cleaned dataset to CSV for visualisation

In [140]:
df.to_csv('../data/clean_online_retail.csv', index=False)